# 컨테이너 기본 개념 및 실습

> **실습 환경**: 이 노트북은 Dev Container(GitHub Codespaces 또는 VS Code)에서 실행하도록 구성되어 있습니다. Docker, Azure CLI 등 필요한 도구가 자동으로 설치되어 있습니다.

이 노트북에서는 컨테이너의 기본 개념, 컨테이너 이미지 빌드 및 실행, 그리고 Azure Container Registry(ACR)로 이미지를 푸시하는 방법을 실습합니다.

## 1. 컨테이너란 무엇인가?
컨테이너는 애플리케이션과 그 실행 환경을 하나의 패키지로 묶어 어디서나 일관되게 실행할 수 있도록 해주는 기술입니다.

### 컨테이너의 주요 이점
- **이식성**: 어디서나 동일하게 실행 가능
- **경량화**: 가상머신보다 리소스 사용이 적음
- **빠른 배포 및 확장**: 이미지 기반으로 빠르게 배포 및 확장 가능
- **격리성**: 애플리케이션 간 충돌 방지

### VM vs 컨테이너 아키텍처 상세 비교

가상머신(VM)과 컨테이너는 모두 애플리케이션을 격리하여 실행하는 기술이지만, 구조와 동작 방식에 차이가 있습니다.

#### 1. VM 아키텍처

아래는 VM 기반 아키텍처의 구조를 나타낸 다이어그램입니다.

```
[앱1]         [앱2]
   |              |
[게스트 OS1]  [게스트 OS2]
   \           /
   [하이퍼바이저]
       |
   [호스트 OS]
       |
   [하드웨어]
```

- 각 VM은 별도의 게스트 OS를 포함하므로 무겁고, 리소스 사용이 많습니다.
- 하이퍼바이저가 하드웨어와 게스트 OS 사이에서 가상화를 담당합니다.

#### 2. 컨테이너 아키텍처

컨테이너 기반 아키텍처의 구조는 다음과 같습니다.

```
   [앱1]      [앱2]
     |           |
[컨테이너1] [컨테이너2]
   |         |
[컨테이너 엔진]
     |
[호스트 OS]
     |
[하드웨어]
```

- 컨테이너는 호스트 OS의 커널을 공유하며, 별도의 OS가 필요하지 않아 경량화되어 있습니다.
- 컨테이너 엔진(예: Docker)이 컨테이너의 실행과 관리를 담당합니다.

#### 3. 비교 요약

| 항목         | VM                                 | 컨테이너                        |
|--------------|-------------------------------------|---------------------------------|
| OS           | 각 VM마다 별도 게스트 OS 필요        | 호스트 OS 커널 공유             |
| 리소스 사용  | 무거움, 오버헤드 큼                  | 경량, 오버헤드 적음             |
| 시작 속도    | 느림                                | 빠름                            |
| 이식성       | 제한적                               | 매우 높음                       |
| 격리 수준    | 강력한 격리(보안성 높음)             | 프로세스 수준 격리              |

### 컨테이너 아키텍처
컨테이너는 호스트 OS의 커널을 공유하며, 각 컨테이너는 독립적으로 실행됩니다. 대표적인 컨테이너 엔진으로는 Docker가 있습니다.

## 2. 실습 환경 설정

> **참고:** 이 실습 가이드는 [Spring 공식 Docker 가이드](https://spring.io/guides/gs/spring-boot-docker)를 기반으로 작성되었습니다. 보다 자세한 내용이나 최신 정보를 원한다면 해당 링크를 참고해 주세요.

> **Dev Container 환경**: GitHub Codespaces 또는 VS Code Dev Container를 사용하면 Python 가상환경, Azure CLI, Docker, kubectl 등 모든 도구가 자동으로 설치되고 설정됩니다.

### 2.1 Python 가상환경 확인

Dev Container가 시작될 때 Python 가상환경(`.venv`)과 Jupyter 커널이 자동으로 생성됩니다. 아래 셀을 실행하여 환경을 확인하세요.

#### Python 환경 자동 설정 완료

Dev Container 시작 시 다음 작업이 자동으로 완료되었습니다:
- ✅ Python 가상환경 생성 (`.venv`)
- ✅ Jupyter 커널 등록 (`Python (.venv)`)
- ✅ 필수 패키지 설치 (`requirements.txt`)
- ✅ Spring Boot 프로젝트 빌드

> **참고**: 커널이 `Python (.venv)`로 설정되어 있는지 확인하세요. VS Code 우측 상단에서 커널을 변경할 수 있습니다.

#### 설치된 패키지 확인

`requirements.txt`에 정의된 모든 패키지가 자동으로 설치되었습니다. 아래 셀을 실행하여 확인하세요.

In [ ]:
# 설치된 패키지 확인
import subprocess
import sys

print("📦 설치된 주요 패키지:")
print("=" * 50)

packages = ['ipykernel', 'jupyter', 'notebook', 'azure-cli-core', 'azure-mgmt-containerregistry', 'azure-mgmt-containerservice', 'requests']

for pkg in packages:
    result = subprocess.run([sys.executable, '-m', 'pip', 'show', pkg], capture_output=True, text=True)
    if result.returncode == 0:
        for line in result.stdout.split('\n'):
            if line.startswith('Name:') or line.startswith('Version:'):
                print(line)
        print("-" * 50)
    else:
        print(f"⚠️  {pkg}: 설치되지 않음")
        print("-" * 50)

print("\n✅ 모든 패키지가 정상적으로 설치되었습니다!")

> **참고**: Dev Container를 사용하면 가상환경이 자동으로 활성화됩니다. 새 터미널을 열어도 자동으로 `.venv`가 활성화됩니다.

### 2.2 개발 도구 버전 확인

Dev Container에 자동으로 설치된 개발 도구들의 버전을 확인합니다.

In [ ]:
%%bash
# Maven 버전 확인
mvn -v

# Java 버전 확인
java -version

# Docker 버전 확인
docker --version

# Azure CLI 버전 확인
az --version

## 3. Spring Boot 애플리케이션 준비

이 섹션에서는 이미 준비된 Spring Boot 애플리케이션을 확인하고, 소스코드부터 컨테이너 이미지 빌드, 실행까지의 전체 과정을 단계별로 실습합니다.

> 💡 **참고**: 이 레포지토리에는 이미 완성된 Spring Boot 프로젝트(`springboot-docker-demo`)가 포함되어 있습니다. 새로 생성할 필요 없이 기존 프로젝트를 사용하여 실습을 진행합니다.

### 3.1 프로젝트 디렉터리 확인

먼저 프로젝트 디렉터리가 존재하는지 확인합니다.

In [ ]:
%%bash
# 프로젝트 디렉터리 확인
if [ -d "springboot-docker-demo" ]; then
    echo "✓ springboot-docker-demo 디렉터리가 존재합니다."
    ls -la springboot-docker-demo/
else
    echo "✗ springboot-docker-demo 디렉터리를 찾을 수 없습니다."
    exit 1
fi

### 3.2 프로젝트 구조 확인

Spring Boot 프로젝트의 디렉터리 구조를 확인합니다.

In [ ]:
%%bash
cd springboot-docker-demo
echo "=== 프로젝트 구조 ==="
echo ""
echo "📁 springboot-docker-demo/"
echo "  ├── pom.xml"
echo "  ├── Dockerfile"
echo "  ├── src/"
echo "  │   ├── main/"
echo "  │   │   ├── java/com/example/springbootdocker/"
echo "  │   │   │   └── App.java"
echo "  │   │   └── resources/"
echo "  │   └── test/"
echo "  │       └── java/"
echo "  └── target/"
echo "      └── demo-0.0.1-SNAPSHOT.jar (빌드 후 생성)"
echo ""
echo "=== 주요 파일 확인 ==="
ls -lh pom.xml Dockerfile src/main/java/com/example/springbootdocker/App.java 2>/dev/null || echo "일부 파일을 찾을 수 없습니다."

### 3.3 pom.xml 확인

Maven 빌드 설정 파일인 `pom.xml`의 내용을 확인합니다. Spring Boot 3.5.6과 Java 21을 사용합니다.

In [ ]:
%%bash
cat springboot-docker-demo/pom.xml

### 3.4 Application.java 확인

Spring Boot 애플리케이션의 메인 클래스를 확인합니다. 이 애플리케이션은 루트 경로(`/`)에서 간단한 메시지를 반환하는 REST API를 제공합니다.

In [ ]:
%%bash
cat springboot-docker-demo/src/main/java/com/example/springbootdocker/App.java

## 4. 컨테이너 이미지 빌드 및 테스트

### 4.1 Dockerfile 작성

프로젝트 루트 디렉터리(`springboot-docker-demo`)에 아래 내용으로 `Dockerfile`을 생성하세요.

> **플랫폼 중립성**: 이 Dockerfile은 멀티 스테이지 빌드를 사용하며, `eclipse-temurin` 공식 이미지는 AMD64(Intel/AMD), ARM64(Apple Silicon) 등 여러 아키텍처를 자동으로 지원합니다. Docker가 실행 중인 플랫폼에 맞는 이미지를 자동으로 선택합니다.

#### Dockerfile 내용 예시

```dockerfile
# 멀티 스테이지 빌드 - 빌드 단계
FROM maven:3.9-eclipse-temurin-21 AS build
WORKDIR /app
COPY pom.xml .
COPY src ./src
RUN mvn clean package -Dmaven.test.skip=true

# 멀티 스테이지 빌드 - 실행 단계
FROM eclipse-temurin:21-jre

# 비-루트 사용자 생성 (Debian 기반)
RUN groupadd -r spring && useradd -r -g spring spring
USER spring:spring

WORKDIR /app
COPY --from=build /app/target/*.jar app.jar

EXPOSE 8080
ENTRYPOINT ["java", "-jar", "app.jar"]
```

아래 셀을 실행하면 이 내용으로 Dockerfile이 자동 생성됩니다.

In [ ]:
%%bash
cat > springboot-docker-demo/Dockerfile << 'EOF'
# 멀티 스테이지 빌드 - 빌드 단계
FROM maven:3.9-eclipse-temurin-21 AS build
WORKDIR /app
COPY pom.xml .
COPY src ./src
RUN mvn clean package -Dmaven.test.skip=true

# 멀티 스테이지 빌드 - 실행 단계
FROM eclipse-temurin:21-jre

# 비-루트 사용자 생성 (Debian 기반)
RUN groupadd -r spring && useradd -r -g spring spring
USER spring:spring

WORKDIR /app
COPY --from=build /app/target/*.jar app.jar

EXPOSE 8080
ENTRYPOINT ["java", "-jar", "app.jar"]
EOF

echo "✅ Dockerfile 생성 완료"
cat springboot-docker-demo/Dockerfile

### 4.2 Docker 이미지 빌드

GitHub Codespaces에는 Docker가 이미 설치되어 있습니다. 아래 명령어로 컨테이너 이미지를 빌드하세요.

> **멀티 스테이지 빌드**: Dockerfile 내부에서 Maven 빌드가 자동으로 실행되므로, 별도로 JAR 파일을 빌드할 필요가 없습니다.

> **중요**: AKS는 AMD64 아키텍처를 사용하므로, M1/M2 Mac에서도 `--platform linux/amd64` 옵션을 사용하여 호환되는 이미지를 빌드합니다.

In [ ]:
%%bash
cd springboot-docker-demo
docker build --platform linux/amd64 -t myapp:latest .

### 4.3 빌드된 이미지 확인

빌드가 완료되면 이미지 목록을 확인할 수 있습니다.

In [ ]:
!docker images

## 5. 컨테이너 실행 실습

빌드한 이미지를 기반으로 컨테이너를 실행하고 테스트합니다.

> **GitHub Codespaces**: Codespaces에서는 포트 포워딩이 자동으로 설정되어, 실행 중인 컨테이너의 포트에 브라우저로 접근할 수 있습니다.

In [ ]:
!docker ps

In [ ]:
!docker run -d -p 8080:8080 --name myapp myapp:latest

### 애플리케이션 테스트

GitHub Codespaces에서는 포트 포워딩이 자동으로 설정됩니다. VS Code 하단의 "PORTS" 탭에서 8080 포트를 확인하고 브라우저로 접근할 수 있습니다.

또는 아래 명령어로 터미널에서 테스트할 수 있습니다.

In [ ]:
!curl http://localhost:8080

## 6. Azure Container Registry(ACR) 생성 및 이미지 푸시

Azure에 컨테이너 이미지를 저장하려면 ACR을 사용합니다. 이 섹션에서는 ACR을 생성하고 빌드한 이미지를 푸시하는 전체 과정을 실습합니다.

### 6.1 Azure CLI 로그인

GitHub Codespaces에는 Azure CLI가 이미 설치되어 있습니다. 먼저 Azure에 로그인합니다.

In [ ]:
# 공통 설정 파일 import
from config import *

# 현재 설정 확인
print_config()

In [ ]:
import subprocess
import json

# Azure 로그인 (항상 새로 로그인)
print("Azure 로그인 중...")
subprocess.run(["az", "login"], check=True)
print("✅ Azure 로그인 완료\n")

# 모든 구독 목록 조회
accounts_result = subprocess.run(
    ["az", "account", "list"],
    capture_output=True,
    text=True,
    check=True
)

accounts = json.loads(accounts_result.stdout)

# 현재 기본 구독 표시
current = next((acc for acc in accounts if acc.get('isDefault')), None)
if current:
    print(f"\n현재 기본 구독: {current['name']}")
    print(f"\n💡 다른 구독을 사용하려면 터미널에서 다음 명령을 실행하세요:")
    print(f"   az account set --subscription \"<구독 이름 또는 ID>\"")

### 6.2 Azure 리소스 그룹 생성

ACR을 생성하기 전에 리소스 그룹을 만들어야 합니다. 리소스 그룹은 관련된 Azure 리소스를 논리적으로 묶어주는 컨테이너 역할을 합니다.

In [ ]:
import subprocess

# config.py의 변수 사용
print(f"📝 리소스 그룹 생성 설정:")
print(f"   이름: {RESOURCE_GROUP}")
print(f"   위치: {LOCATION}")
print()

# 리소스 그룹 생성
result = subprocess.run([
    "az", "group", "create",
    "--name", RESOURCE_GROUP,
    "--location", LOCATION
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ 리소스 그룹 생성 완료!")
    print()
    # 생성된 리소스 그룹 정보 확인
    subprocess.run([
        "az", "group", "show",
        "--name", RESOURCE_GROUP,
        "--query", "{Name:name, Location:location, ProvisioningState:properties.provisioningState}",
        "--output", "table"
    ])
else:
    print("❌ 리소스 그룹 생성 실패")
    print(result.stderr)

### 6.3 ACR(Azure Container Registry) 생성

이제 컨테이너 이미지를 저장할 ACR을 생성합니다.

> **참고**: ACR 이름은 전역적으로 고유해야 하며, 5-50자의 영숫자만 사용 가능합니다.

In [ ]:
import subprocess
import time

# ACR 이름 생성 (타임스탬프로 고유한 이름 생성)
ACR_NAME = f"myacr{int(time.time())}"
SKU = "Basic"

print(f"📝 ACR 생성 설정:")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   ACR 이름: {ACR_NAME}")
print(f"   SKU: {SKU}")
print()

# ACR 생성
result = subprocess.run([
    "az", "acr", "create",
    "--resource-group", RESOURCE_GROUP,
    "--name", ACR_NAME,
    "--sku", SKU,
    "--admin-enabled", "true"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ ACR 생성 완료!")
    print(f"   ACR 이름: {ACR_NAME}")
    print(f"   로그인 서버: {ACR_NAME}.azurecr.io")
    print()
    print("=" * 80)
    print("⚠️  중요: 다음 단계를 수행하세요!")
    print("=" * 80)
    print(f"1. config.py 파일을 열고 ACR_NAME 값을 업데이트하세요:")
    print(f"   ACR_NAME = \"{ACR_NAME}\"")
    print()
    print("2. 이렇게 하면 02, 03번 노트북에서 동일한 ACR을 자동으로 사용할 수 있습니다.")
    print("=" * 80)
else:
    print("❌ ACR 생성 실패")
    print(result.stderr)

### 6.4 ACR 정보 확인

생성된 ACR의 상세 정보를 확인합니다.

In [ ]:
import subprocess

# 위 셀에서 생성된 ACR_NAME 변수 사용
result = subprocess.run([
    "az", "acr", "show",
    "--name", ACR_NAME,
    "--query", "{Name:name, LoginServer:loginServer, SKU:sku.name, Location:location}",
    "--output", "table"
], capture_output=True, text=True)

print(result.stdout)

### 6.5 ACR에 로그인

Docker가 ACR에 이미지를 푸시할 수 있도록 ACR에 로그인합니다.

In [ ]:
import subprocess

# ACR에 로그인
result = subprocess.run([
    "az", "acr", "login",
    "--name", ACR_NAME
], capture_output=True, text=True)

print(result.stdout)
print("")
print("✅ ACR 로그인 성공!")

### 6.6 이미지 태깅

ACR로 이미지를 푸시하기 전에 ACR 주소로 태깅해야 합니다.

In [ ]:
import subprocess

# config.py의 헬퍼 함수 사용
ACR_LOGIN_SERVER = get_acr_login_server(ACR_NAME)
IMAGE_NAME = f"{ACR_LOGIN_SERVER}/{APP_NAME}:{IMAGE_TAG}"

# 이미지 태깅
result = subprocess.run([
    "docker", "tag",
    f"{APP_NAME}:{IMAGE_TAG}",
    IMAGE_NAME
], capture_output=True, text=True)

print(result.stdout)
print(f"✅ 이미지 태깅 완료: {IMAGE_NAME}")
print()
print("💡 사용된 config.py 변수:")
print(f"   ACR_NAME: {ACR_NAME}")
print(f"   APP_NAME: {APP_NAME}")
print(f"   IMAGE_TAG: {IMAGE_TAG}")

### 6.7 이미지 푸시

태깅된 이미지를 ACR로 푸시합니다.

In [ ]:
import subprocess

# config.py의 헬퍼 함수 사용하여 전체 이미지 이름 생성
IMAGE_NAME = get_full_image_name(ACR_NAME, APP_NAME, IMAGE_TAG)

# 이미지 푸시
result = subprocess.run([
    "docker", "push",
    IMAGE_NAME
], capture_output=True, text=True)

print(result.stdout)
print("")
print(f"✅ 이미지 푸시 완료: {IMAGE_NAME}")
print()
print("💡 다음 단계:")
print(f"   1. config.py 파일의 ACR_NAME을 '{ACR_NAME}'로 업데이트하세요")
print("   2. 02번 노트북(02-aks-hands-on.ipynb)에서 이 이미지를 AKS에 배포할 수 있습니다")

### 6.8 ACR 이미지 목록 확인

ACR에 푸시된 이미지를 확인합니다.

In [ ]:
import subprocess

# ACR에 푸시된 이미지 목록 확인
result = subprocess.run([
    "az", "acr", "repository", "list",
    "--name", ACR_NAME,
    "--output", "table"
], capture_output=True, text=True)

print(result.stdout)
print("")
print("✅ ACR에 저장된 이미지 목록 확인 완료!")

## 7. 정리

이 노트북에서는 컨테이너의 기본 개념부터 실제 Spring Boot 애플리케이션을 컨테이너 이미지로 빌드하고, 실행 및 Azure Container Registry(ACR)로 푸시하는 전체 과정을 GitHub Codespaces 환경에서 실습했습니다.

### 실습한 내용 요약
- ✅ Python 가상환경 생성 및 패키지 설치
- ✅ 컨테이너 개념 및 아키텍처 이해
- ✅ Spring Boot 애플리케이션 작성
- ✅ 플랫폼 중립적 Dockerfile 작성 및 이미지 빌드
- ✅ 컨테이너 실행 및 테스트
- ✅ Azure 리소스 그룹 및 ACR 생성
- ✅ Azure Container Registry에 이미지 푸시


> **다음 단계**: `02-aks-hands-on.ipynb`에서 ACR에 푸시한 이미지를 AKS(Azure Kubernetes Service) 클러스터에 배포하는 방법을 실습합니다.